In [3]:
!pip install cohere


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
from pprint import pprint
import cohere
import pandas as pd

In [5]:
COHERE_MODEL = 'command-r-plus' 
co = cohere.Client('vQPugCWtSolhoXDEUiXrqQEP9ocJr1B1WT1t3rls')

In [6]:
message = "who is yoyo honey singh?"
response = co.chat(
  message=message,
  model="command",
  temperature=0.3
)
answer = response.text
pprint(answer)

('Yo Yo Honey Singh is a Indian rapper, singer, songwriter and music producer '
 'who gained popularity in Indian pop music and Bollywood playback singing. He '
 'began his career in 2003 and has since then released several successful '
 'songs that have topped the charts. His songs are usually a mix of Hindi and '
 'English words, and his beats are usually upbeat and high-tempo, comprising '
 'electro hop and dance music.')


In [7]:
documents = [
    {
        "title": "Supply Chain Regulations",
        "body": """
        Supplier Compliance - All suppliers must adhere to local, state, and federal laws governing supply chain practices, including product safety, labor laws, and environmental regulations. For specific guidelines, refer to Section 5b for more details.
        Supplier Responsibility - Suppliers are responsible for ensuring that all goods and materials comply with applicable laws and meet the quality standards set by the purchasing company (SCC 12.34.567). Non-compliance may result in penalties, including fines and contract termination.
        Restricted Goods - Some goods, particularly hazardous materials, may be subject to import/export restrictions. See the restricted goods guidelines in Section 7c for a list of these items.
        Child Labor - It is illegal for any supplier to employ workers under the age of 14. For detailed child labor regulations and penalties, see Section 4a (SCC 12.34.578).
        Transport and Delivery - All suppliers must ensure that goods are transported according to the safety regulations outlined in SCC 12.34.890. Delivery schedules must also comply with these regulations, and suppliers may be fined for any safety violations.
        Traceability - Suppliers must maintain records of product origin and material sourcing. In the event of a product recall, these records will be used to trace the defective goods to their source (SCC 12.34.999).
        Environmental Standards - Suppliers must follow all relevant environmental standards. Failure to comply with these standards could result in penalties under Section 9b. For more information, refer to the Environmental Protection Agency's guidelines.
        Supplier Contracts - All contracts with suppliers must include clauses related to compliance with labor, safety, and environmental laws. For a full list of required contract clauses, refer to Section 6b.
        """
    },
    {
        "title": "Child Labor Laws",
        "body": "Suppliers are prohibited from employing workers under the age of 14 in any part of the supply chain. Violations of this law may result in severe penalties. Refer to Section 4a for specific penalties and guidelines on child labor restrictions.",
    },
    {
        "title": "Section 4a",
        "body": """Child Labor Restrictions: All suppliers must comply with child labor laws that prohibit the employment of minors under the age of 14.
        Violations will incur the following penalties:
        1. First offense: $5,000 fine and immediate suspension of contract.
        2. Second offense: $10,000 fine and permanent ban from future contracts.
        3. Suppliers found to be engaging in child labor practices in foreign countries will be subject to penalties under the international labor laws section (IL 12.34.678).

        For more information on child labor laws, please visit Section 9b of the International Labor Code.
        """
    },
    {
        "title": "Section 5b",
        "body": """Supplier Compliance: Suppliers must follow all local, state, and federal regulations governing supply chain practices, including the following:
        1. Product Safety: Suppliers must ensure that all products meet safety standards outlined in SCC 12.34.789.
        2. Labor Laws: Suppliers must adhere to local labor laws regarding wages, working hours, and working conditions (SCC 12.34.567).
        3. Environmental Compliance: Suppliers are required to comply with environmental protection regulations, including waste disposal and resource usage limits.

        Penalties for non-compliance:
        - First violation: $2,500 fine and warning.
        - Second violation: $5,000 fine and review of contract status.
        - Third violation: Termination of contract and legal action.
        """
    },
    {
        "title": "Section 7c",
        "body": """Restricted Goods: Certain products and materials may not be imported or exported without special permits, including:
        1. Hazardous chemicals.
        2. Goods subject to trade embargoes.
        3. Wildlife products subject to conservation laws.

        For a complete list of restricted goods, refer to the International Trade Regulations (ITR 12.34.567).
        """
    }
]

db = pd.DataFrame(documents)
db["combined"] = "Title: " + db["title"] + "\n" + "Body: " + db["body"]

embeddings = co.embed(
    texts=db.combined.tolist(), model="embed-english-v3.0", input_type="search_document"
)
db["embeddings"] = embeddings.embeddings


In [8]:
db.head()

,title,body,combined,embeddings
0,Supply Chain Regulations,\n Supplier Compliance - All suppliers ...,Title: Supply Chain Regulations\nBody: \n ...,"[-0.0039405823, -0.013084412, -0.01940918, -0...."
1,Child Labor Laws,Suppliers are prohibited from employing worker...,Title: Child Labor Laws\nBody: Suppliers are p...,"[-0.003332138, 0.010688782, -0.0009689331, -0...."
2,Section 4a,Child Labor Restrictions: All suppliers must c...,Title: Section 4a\nBody: Child Labor Restricti...,"[0.016113281, 0.0026474, 0.001496315, -0.05053..."
3,Section 5b,Supplier Compliance: Suppliers must follow all...,Title: Section 5b\nBody: Supplier Compliance: ...,"[0.017349243, 0.00869751, -0.020874023, -0.080..."
4,Section 7c,Restricted Goods: Certain products and materia...,Title: Section 7c\nBody: Restricted Goods: Cer...,"[-0.034698486, 0.011146545, -0.026260376, -0.0..."


In [9]:
from transformers import DPRQuestionEncoder, DPRContextEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer
import torch

question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

def retrieve_documents(query: str, n=1) -> dict:
    query_emb = question_encoder(**question_tokenizer(query, return_tensors="pt")).pooler_output
    doc_embeddings = context_encoder(**context_tokenizer(db['combined'].tolist(), return_tensors="pt", padding=True, truncation=True)).pooler_output
    similarity_scores = torch.matmul(query_emb, doc_embeddings.T).squeeze(0).cpu().detach().numpy()
    top_indices = similarity_scores.argsort()[::-1][:n]
    top_matches = db.iloc[top_indices]
    return {"top_matched_document": top_matches.combined}



functions_map = {
    "retrieve_documents": retrieve_documents,
}

tools = [
    {
        "name": "retrieve_documents",
        "description": "given a query, retrieve documents from a database to answer user's question",
        "parameter_definitions": {
            "query": {"description": "query", "type": "str", "required": True}
        },
    }
]

c:\Users\agrvi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model tha

In [10]:
def simple_rag(query, db):
    """
    Given user's query, retrieve top documents and generate response using documents parameter.
    """
    top_matched_document = retrieve_documents(query)["top_matched_document"]
    print("top_matched_document", top_matched_document)
    output = co.chat(
        message=query, model=COHERE_MODEL, documents=[top_matched_document]
    )
    return output.text

In [11]:
def cohere_agent(
    message: str,
    preamble: str,
    tools: list[dict],
    force_single_step=False,
    verbose: bool = False,
    temperature: float = 0.3,) -> str:

    """
    Function to handle multi-step tool use api.

    Args:
        message (str): The message to send to the Cohere AI model.
        preamble (str): The preamble or context for the conversation.
        tools (list of dict): List of tools to use in the conversation.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        str: The final response from the call.
    """

    counter = 1
    response = co.chat(
        model=COHERE_MODEL,
        message=message,
        preamble=preamble,
        tools=tools,
        force_single_step=force_single_step,
        temperature=temperature,
    )

    if verbose:
        print(f"\nrunning 0th step.")
        print(response.text)

    while response.tool_calls:
        tool_results = []

        if verbose:
            print(f"\nrunning {counter}th step.")

        for tool_call in response.tool_calls:
            output = functions_map[tool_call.name](**tool_call.parameters)
            outputs = [output]
            tool_results.append({"call": tool_call, "outputs": outputs})

            if verbose:
                print(
                    f"= running tool {tool_call.name}, with parameters: \n{tool_call.parameters}"
                )
                print(f"== tool results:")
                pprint(output)

        response = co.chat(
            model=COHERE_MODEL,
            message="",
            chat_history=response.chat_history,
            preamble=preamble,
            tools=tools,
            force_single_step=force_single_step,
            tool_results=tool_results,
            temperature=temperature,
        )

        if verbose:
            print(response.text)
            counter += 1

    return response.text



In [12]:
question1 = "Is there a law preventing suppliers from using child labor?"

In [13]:
output = simple_rag(question1, db)
print(output)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


top_matched_document 1    Title: Child Labor Laws\nBody: Suppliers are p...
Name: combined, dtype: object
Yes, there is a law that prohibits suppliers from employing workers under the age of 14 in any part of the supply chain. Violations may result in severe penalties.


In [14]:
preamble = """
You are an expert assistant that helps users answers question about legal documents and policies.
Use the provided documents to answer questions about an employee's specific situation.
"""

output = cohere_agent(question1, preamble, tools, verbose=True)
pprint(output)


running 0th step.
I will search the database for 'child labour' to see if there is a law preventing suppliers from using child labour.

running 1th step.
= running tool retrieve_documents, with parameters: 
{'query': 'child labour'}
== tool results:
{'top_matched_document': 1    Title: Child Labor Laws\nBody: Suppliers are p...
Name: combined, dtype: object}
Yes, there is a law preventing suppliers from using child labour. Suppliers are prohibited from employing workers under the age of 14 in any part of the supply chain. Violations of this law may result in severe penalties.
('Yes, there is a law preventing suppliers from using child labour. Suppliers '
 'are prohibited from employing workers under the age of 14 in any part of the '
 'supply chain. Violations of this law may result in severe penalties.')


In [15]:
question2 = """I operate a supply chain business in DuPont, and I’m sourcing materials from multiple international locations.
              Do I need to comply with both local and international labor and environmental laws,
              and what are the penalties if a supplier in a foreign country is found violating child labor laws?
              """

In [16]:
output = simple_rag(question2, db)
pprint(output)

top_matched_document 1    Title: Child Labor Laws\nBody: Suppliers are p...
Name: combined, dtype: object
('Suppliers are prohibited from employing workers under the age of 14 in any '
 'part of the supply chain. Violations of this law may result in severe '
 'penalties. Unfortunately, that is all the information I have on the topic.')


In [17]:
preamble = """
You are an expert assistant that helps users answers question about legal documents and policies.
Use the provided documents to answer questions about an employee's specific situation.
"""
output = cohere_agent(question2, preamble, tools, verbose=True)


running 0th step.
I will search for 'comply with local and international laws' and 'penalties for child labour' to answer the user's question.

running 1th step.
= running tool retrieve_documents, with parameters: 
{'query': 'comply with local and international laws'}
== tool results:
{'top_matched_document': 3    Title: Section 5b\nBody: Supplier Compliance: ...
Name: combined, dtype: object}
= running tool retrieve_documents, with parameters: 
{'query': 'penalties for child labour'}
== tool results:
{'top_matched_document': 1    Title: Child Labor Laws\nBody: Suppliers are p...
Name: combined, dtype: object}
Suppliers must follow all local, state, and federal regulations governing supply chain practices. This includes labour laws and environmental compliance. 

If a supplier is found to be violating child labour laws, this may result in severe penalties. The specific penalties are outlined in Section 4a.


In [18]:
def reference_extractor(query: str, documents: list[str]) -> str:
    """
    Given a query and document, find references to other documents.
    """
    prompt = f"""
    # instruction
    Does the reference document mention any other documents? If so, list them.
    If not, return empty string.

    # user query
    {query}

    # retrieved documents
    {documents}
    """
    return co.chat(message=prompt, model=COHERE_MODEL, preamble=None).text


def retrieve_documents(query: str, n=1) -> dict:
    """
    Function to retrieve the most relevant documents for a given query.
    It also returns other references mentioned in the top-matched documents.
    """
    query_input = question_tokenizer(query, return_tensors="pt")
    query_emb = question_encoder(**query_input).pooler_output

    doc_texts = db['combined'].tolist()
    doc_inputs = context_tokenizer(doc_texts, return_tensors="pt", padding=True, truncation=True)
    doc_embeddings = context_encoder(**doc_inputs).pooler_output

    similarity_scores = torch.matmul(query_emb, doc_embeddings.T).squeeze(0).cpu().detach().numpy()

    top_indices = similarity_scores.argsort()[::-1][:n]
    top_matches = db.iloc[top_indices]

    # Extract references from the top-matched documents
    other_references = reference_extractor(query, top_matches.combined.tolist())

    return {
        "top_matched_document": top_matches.combined,
        "other_references_to_query": other_references,
    }



functions_map = {
    "retrieve_documents": retrieve_documents,
}

tools = [
    {
        "name": "retrieve_documents",
        "description": "given a query, retrieve documents from a database to answer user's question. It also finds references to other documents that should be leveraged to retrieve more documents",
        "parameter_definitions": {
            "query": {
                "description": "user's question or question or name of other document sections or references.",
                "type": "str",
                "required": True,
            }
        },
    }
]

In [21]:
preamble2 = """# Instruction
You are an expert assistant that helps users answer questions about legal documents and policies.

Please follow these steps:
1. Using user's query, use `retrieve_documents` tool to retrieve the most relevant document from the database.
2. If you see `other_references_to_query` in the tool result, search the mentioned referenced using `retrieve_documents(<other reference="">)` tool to retrieve more documents.
3. Keep trying until you find the answer.
4. Answer with yes or no as much as you can to answer the question directly.
"""

output1 = cohere_agent(question2, preamble2, tools, verbose=True)



running 0th step.
I will search for 'international labour and environmental laws' and 'child labour law penalties' to answer the user's query.

running 1th step.
= running tool retrieve_documents, with parameters: 
{'query': 'international labour and environmental laws'}
== tool results:
{'other_references_to_query': 'International Trade Regulations (ITR 12.34.567)',
 'top_matched_document': 4    Title: Section 7c\nBody: Restricted Goods: Cer...
Name: combined, dtype: object}
= running tool retrieve_documents, with parameters: 
{'query': 'child labour law penalties'}
== tool results:
{'other_references_to_query': 'Section 4a',
 'top_matched_document': 1    Title: Child Labor Laws\nBody: Suppliers are p...
Name: combined, dtype: object}
I have found that suppliers are prohibited from employing workers under the age of 14 in any part of the supply chain. Violations of this law may result in severe penalties. I will now search for the specific penalties in Section 4a.

running 2th step.


In [22]:
def legal_advisor(query: str, documents: list[str]) -> str:
    """
    Provides legal advice based on the query and relevant documents.
    It formulates a response considering legal constraints and guidelines.

    Args:
    - query: User's legal-related question.
    - documents: List of relevant documents retrieved.

    Returns:
    - Legal advice based on the provided documents.
    """
    prompt = f"""
    # Instruction:
    Provide legal advice based on the user's question. Only use information from the provided documents and reference applicable laws.

    # User Query:
    {query}

    # Relevant Documents:
    {documents}

    # Output:
    Give clear legal advice that directly addresses the user's query.
    """
    return co.chat(message=prompt, model=COHERE_MODEL, preamble=None).text

def regulations(query: str, documents: list[str]) -> str:
    """
    Provides information on regulations based on the query and relevant documents.
    It extracts specific sections and compliance requirements.

    Args:
    - query: User's question related to regulations.
    - documents: List of relevant documents retrieved.

    Returns:
    - A response focusing on regulatory compliance, highlighting relevant sections and requirements.
    """
    prompt = f"""
    # Instruction:
    Provide detailed information on the regulations relevant to the user's query. Use only the provided documents and reference applicable sections.

    # User Query:
    {query}

    # Relevant Documents:
    {documents}

    # Output:
    Focus on regulatory compliance, highlighting the sections or guidelines that are most relevant to the query.
    """
    return co.chat(message=prompt, model=COHERE_MODEL, preamble=None).text



def hallucination_detector(query: str, response: str, documents: list[str]) -> str:
    """
    Detects hallucinations by checking if the generated response contains information not supported by the provided documents.

    Args:
    - query: The original user query.
    - response: The generated response based on the query.
    - documents: The list of retrieved documents.

    Returns:
    - A message indicating whether hallucinations were detected, and if so, highlights unsupported content.
    """
    prompt = f"""
    # Instruction:
    Check the provided response to see if it contains information that is not supported or referenced in the documents.

    # Query:
    {query}

    # Retrieved Documents:
    {documents}

    # Generated Response:
    {response}

    # Output format:
    If the response contains hallucinations, list the hallucinated parts and explain why they are unsupported. If everything is accurate, return "No hallucinations detected."
    """
    return co.chat(message=prompt, model=COHERE_MODEL, preamble=None).text


def reference_extractor(query: str, documents: list[str]) -> str:
    """
    Given a query and documents, find references to other documents.
    """
    prompt = f"""
    # Instruction:
    Does the reference document mention any other documents? If so, list them. If not, return an empty string.

    # User Query:
    {query}

    # Retrieved Documents:
    {documents}
    """
    return co.chat(message=prompt, model=COHERE_MODEL, preamble=None).text


def retrieve_documents(query: str, n=1) -> dict:
    """
    Function to retrieve the most relevant documents for a given query.
    It also returns other references mentioned in the top-matched documents.
    """
    query_input = question_tokenizer(query, return_tensors="pt")
    query_emb = question_encoder(**query_input).pooler_output

    doc_texts = db['combined'].tolist()
    doc_inputs = context_tokenizer(doc_texts, return_tensors="pt", padding=True, truncation=True)
    doc_embeddings = context_encoder(**doc_inputs).pooler_output

    similarity_scores = torch.matmul(query_emb, doc_embeddings.T).squeeze(0).cpu().detach().numpy()

    top_indices = similarity_scores.argsort()[::-1][:n]
    top_matches = db.iloc[top_indices]

    # Extract references from the top-matched documents
    other_references = reference_extractor(query, top_matches.combined.tolist())

    return {
        "top_matched_document": top_matches.combined,
        "other_references_to_query": other_references,
    }


def generate_response_with_hallucination_check(query: str, documents: list[str]) -> dict:
    """
    Generates a response for the given query based on the provided documents and checks for hallucinations in the response.
    """
    prompt = f"""
    # Instruction:
    Generate an informative and accurate response to the user's query using only the information available in the retrieved documents.

    # User Query:
    {query}

    # Retrieved Documents:
    {documents}
    """
    response = co.chat(message=prompt, model=COHERE_MODEL, preamble=None).text

    hallucination_check = hallucination_detector(query, response, documents)

    return {
        "generated_response": response,
        "hallucination_check": hallucination_check,
    }


functions_map = {
    "retrieve_documents": retrieve_documents,
    "generate_response_with_hallucination_check": generate_response_with_hallucination_check,
    "legal_advisor": legal_advisor,
    "regulations": regulations,
}


tools = [
    {
        "name": "retrieve_documents",
        "description": "Retrieve documents relevant to a user's question.",
        "parameter_definitions": {
            "query": {
                "description": "The user's question or request for documents.",
                "type": "str",
                "required": True,
            }
        },
    },
    {
        "name": "generate_response_with_hallucination_check",
        "description": "Generates a response based on retrieved documents and checks for hallucinations.",
        "parameter_definitions": {
            "query": {
                "description": "User's query or input question.",
                "type": "str",
                "required": True,
            },
            "documents": {
                "description": "The list of retrieved documents relevant to the user's query.",
                "type": "list[str]",
                "required": True,
            }
        },
    },
    {
        "name": "legal_advisor",
        "description": "Provides legal advice based on relevant documents.",
        "parameter_definitions": {
            "query": {
                "description": "The user's legal question.",
                "type": "str",
                "required": True,
            },
            "documents": {
                "description": "The list of retrieved documents that contain relevant legal information.",
                "type": "list[str]",
                "required": True,
            }
        },
    },
    {
        "name": "regulations",
        "description": "Provides detailed information on regulations relevant to the user's query.",
        "parameter_definitions": {
            "query": {
                "description": "User's question related to regulations or compliance.",
                "type": "str",
                "required": True,
            },
            "documents": {
                "description": "The list of retrieved documents relevant to regulations and compliance.",
                "type": "list[str]",
                "required": True,
            }
        },
    }
]



In [23]:
def cohere_agent1(
    message: str,
    preamble: str,
    tools: list[dict],
    force_single_step=False,
    verbose: bool = False,
    temperature: float = 0.3,
) -> str:
    """
    Handles multi-step tool usage and integrates legal and regulation agents.

    Args:
        message (str): The message or query to process.
        preamble (str): The preamble or context for the conversation.
        tools (list[dict]): The tools available for use in the agent.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.
        temperature (float, optional): Controls the creativity of the responses.

    Returns:
        str: Final response from the agent.
    """

    counter = 1
    chat_history = []  
    response = co.chat(
        model=COHERE_MODEL,
        message=message,
        preamble=preamble,
        tools=tools,
        force_single_step=force_single_step,
        temperature=temperature,
    )

    chat_history.append({"message": message, "response": response.text})  

    if verbose:
        print(f"\nrunning 0th step.")
        print(response.text)

    retrieved_documents = pd.Series(dtype='object')  # Initialize an empty Series to hold documents

    while response.tool_calls:
        tool_results = []

        if verbose:
            print(f"\nrunning {counter}th step.")

        for tool_call in response.tool_calls:
            if tool_call.name == "retrieve_documents":
                output = functions_map[tool_call.name](**tool_call.parameters)
                top_matched_document = output["top_matched_document"]
                other_references = output["other_references_to_query"]

                retrieved_documents = pd.concat([retrieved_documents, top_matched_document])
                tool_results.append({"call": tool_call, "outputs": [output]})

                if verbose:
                    print(f"= running tool {tool_call.name}, with parameters: \n{tool_call.parameters}")
                    pprint(output)

                if other_references:
                    for reference in other_references.split(","):
                        reference_output = functions_map[tool_call.name](query=reference)
                        reference_document = reference_output["top_matched_document"]
                        retrieved_documents = pd.concat([retrieved_documents, reference_document])

            elif tool_call.name in ["legal_advisor", "regulations"]:
                output = functions_map[tool_call.name](**tool_call.parameters)
                tool_results.append({"call": tool_call, "outputs": [output]})
                response = output

                chat_history.append({"message": message, "response": response})

                if verbose:
                    print(f"== {tool_call.name} response:")
                    pprint(output)

            elif tool_call.name == "generate_response_with_hallucination_check":
                output = functions_map[tool_call.name](**tool_call.parameters)
                hallucination_check = output["hallucination_check"]
                tool_results.append({"call": tool_call, "outputs": [output]})

                if "No hallucinations detected" not in hallucination_check:
                    return hallucination_check

        if not retrieved_documents.empty:
            response_with_check = functions_map["generate_response_with_hallucination_check"](
                query=message, documents=retrieved_documents.tolist()
            )
            response = response_with_check["generated_response"]
            hallucination_check = response_with_check["hallucination_check"]

            chat_history.append({"message": message, "response": response})

            if "No hallucinations detected" in hallucination_check:
                return response
            else:
                return hallucination_check

        response = co.chat(
            model=COHERE_MODEL,
            message="",
            chat_history=chat_history,
            preamble=preamble,
            tools=tools,
            force_single_step=force_single_step,
            tool_results=tool_results,
            temperature=temperature,
        )

        chat_history.append({"message": "", "response": response.text})

        if verbose:
            print(response.text)
            counter += 1

    return response.text


In [24]:
preamble2 = """# Instruction
You are an expert assistant that helps users answer questions about legal documents and policies.

Please follow these steps:
1. Using user's query, use `retrieve_documents` tool to retrieve the most relevant document from the database.
2. If you see `other_references_to_query` in the tool result, search the mentioned referenced using `retrieve_documents(<other reference="">)` tool to retrieve more documents.
3. Keep trying until you find the answer.
4. Answer with yes or no as much as you can to answer the question directly.
"""

output2 = cohere_agent1(question2, preamble2, tools, verbose=True)


running 0th step.
I will search for 'international supply chain business compliance' and 'penalties for child labour law violations' to answer the user's query.

running 1th step.
= running tool retrieve_documents, with parameters: 
{'query': 'international supply chain business compliance'}
{'other_references_to_query': '- Section 5b\n'
                              '- SCC 12.34.567\n'
                              '- Section 7c\n'
                              '- Section 4a (SCC 12.34.578)\n'
                              '- SCC 12.34.890\n'
                              '- SCC 12.34.999\n'
                              "- Section 9b (Environmental Protection Agency's "
                              'guidelines)\n'
                              '- Section 6b',
 'top_matched_document': 0    Title: Supply Chain Regulations\nBody: \n     ...
Name: combined, dtype: object}
= running tool retrieve_documents, with parameters: 
{'query': 'penalties for child labour law violations'}
{'other

In [25]:
print(question2)

I operate a supply chain business in DuPont, and I’m sourcing materials from multiple international locations.
              Do I need to comply with both local and international labor and environmental laws,
              and what are the penalties if a supplier in a foreign country is found violating child labor laws?
              


Output After using 2 Agents.

In [26]:
print(output1)

Yes, you need to comply with both local and international labour and environmental laws. If a supplier in a foreign country is found violating child labour laws, the penalties are as follows:
1. First offence: $5,000 fine and immediate suspension of contract.
2. Second offence: $10,000 fine and permanent ban from future contracts.
3. Suppliers found to be engaging in child labour practices in foreign countries will be subject to penalties under the international labour laws section (IL 12.34.678).


Output after using 5 Agents

In [27]:
print(output2)

## Compliance with Laws

As a supply chain business in DuPont, you must ensure compliance with local, state, and federal laws, including labor laws, environmental regulations, and product safety standards. This applies not only to your operations in DuPont but also to your sourcing of materials from international locations. 

## Supplier Responsibility

Your suppliers are responsible for ensuring that the goods and materials they provide comply with the applicable laws and meet the quality standards set by your company. This includes compliance with local labor laws regarding wages, working hours, and working conditions in the countries from which you are sourcing materials. 

## Child Labor Laws

It is illegal for any supplier in your supply chain to employ workers under the age of 14. Violations of child labor laws will result in penalties, and the specific consequences are outlined in Section 4a:

1. First Offense: A fine of $5,000 is imposed, and the contract with the violating sup